## Grid Search with a Keras Model in scikit-learn

### Import some data

In [1]:
import pandas as pd

# Import data from url
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv')

In [2]:
# Peek at the data
data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


### Tuninig batch size and number of epochs
(Note: LSTMs, CNNs and RNNs are sensible to the batch size)

In [3]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [4]:
np.random.seed(42)

In [5]:
# Wrapper function for the keras model
# (Required for KerasClassifier/KerasRegressor of scikit-learn)

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# Split data into features and label
X = data.iloc[:,0:-1]
Y = data.iloc[:,-1:]

In [7]:
# Define parameters for grid search
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
params = dict(batch_size=batch_size, epochs=epochs)

In [8]:
# Define keras model within scikit
model = KerasClassifier(build_fn=create_model, verbose=0)

In [10]:
# Setup grid search and fit different parameters
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=1, verbose=1)
grid_result = grid.fit(X,Y)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  4.4min finished


In [14]:
# Visualize the different results

print("Best score: %f using parameters %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in sorted(zip(means, stds, params)):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best score: 0.696219 using parameters {'batch_size': 20, 'epochs': 100}
0.464146 (0.149314) with: {'batch_size': 80, 'epochs': 50}
0.471969 (0.112846) with: {'batch_size': 100, 'epochs': 10}
0.490222 (0.096743) with: {'batch_size': 80, 'epochs': 10}
0.504563 (0.048982) with: {'batch_size': 60, 'epochs': 10}
0.554107 (0.152387) with: {'batch_size': 40, 'epochs': 100}
0.563233 (0.130443) with: {'batch_size': 40, 'epochs': 50}
0.568449 (0.162544) with: {'batch_size': 80, 'epochs': 100}
0.569752 (0.130811) with: {'batch_size': 10, 'epochs': 50}
0.608866 (0.009164) with: {'batch_size': 100, 'epochs': 50}
0.634941 (0.026417) with: {'batch_size': 60, 'epochs': 50}
0.637549 (0.029777) with: {'batch_size': 10, 'epochs': 10}
0.640156 (0.024296) with: {'batch_size': 100, 'epochs': 100}
0.640156 (0.036383) with: {'batch_size': 20, 'epochs': 50}
0.645372 (0.053173) with: {'batch_size': 40, 'epochs': 10}
0.662321 (0.009175) with: {'batch_size': 20, 'epochs': 10}
0.663625 (0.030565) with: {'batch_siz